In [1]:
import pickle
import matplotlib.pyplot as plt

In [2]:
# load data for hard failure
with open("log_red2.pkl", "rb") as f:
    data = pickle.load(f)


In [13]:
print(len(data[0]), len(data[1]), len(data[2]), len(data[3]))
primary_board_ind = 1
redundant_board_ind = 2

68 37 49 84


In [4]:
data_format = ('board_id', 'pkt_no', 'Tx_time', 'reTx_time', 'RSSI', 'payload')

In [14]:
### Number of Packets received by primary board and redundant board
print('Primary Board:', len(data[primary_board_ind]))
print('Redundant Board:', len(data[redundant_board_ind]))


Primary Board: 37
Redundant Board: 49


In [12]:
data[1]

[(94420780,
  0,
  31,
  31,
  -104,
  (585.0999755859375,
   0.0,
   23.304964065551758,
   1009.3259887695312,
   22.600000381469727,
   39.400001525878906,
   22.80000114440918,
   46.20000076293945,
   22.80000114440918,
   47.0,
   22.899999618530273,
   46.5,
   -157.0,
   0,
   0,
   0,
   94420780)),
 (94420780,
  1,
  61,
  61,
  -103,
  (601.4099731445312,
   0.0,
   22.413785934448242,
   1009.2979736328125,
   22.600000381469727,
   39.20000076293945,
   22.899999618530273,
   46.20000076293945,
   22.899999618530273,
   46.70000076293945,
   22.899999618530273,
   46.400001525878906,
   -44.0,
   0,
   0,
   1,
   94420780)),
 (94420780,
  2,
  91,
  91,
  -94,
  (595.6400146484375,
   0.0,
   22.071962356567383,
   1009.27734375,
   22.700000762939453,
   39.0,
   22.899999618530273,
   45.900001525878906,
   22.899999618530273,
   46.400001525878906,
   22.899999618530273,
   46.10000228881836,
   -44.0,
   0,
   0,
   2,
   94420780)),
 (94420780,
  3,
  121,
  121,
  -

In [15]:
### check if all packets belong to same board
### PB
pb_id = 0
rb_id = 0
for d_i, d in enumerate(data[primary_board_ind]):
    if d_i == 0:
        pb_id = d[0]
        print('PB Id:',pb_id) 
    assert(data[primary_board_ind][0][0]==d[0])
### RB
for d_i, d in enumerate(data[redundant_board_ind]):
    if d_i == 0:
        rb_id = d[0]
        print('RB Id:', rb_id)
    assert(data[redundant_board_ind][0][0]==d[0])

PB Id: 94420780
RB Id: 2750291925


#### Checking the confidence interval (40s)


In [18]:
data_all = data[primary_board_ind] + data[redundant_board_ind]
print(len(data_all))

86


In [17]:
for d in data_all:
    if d[3]-d[2]>0:
        #print(d[1],d[4:6], d[5]-d[4])
        print('packets outside conf. interval', d)

#### PRR of the sensor node as whole (PB+RB)


In [83]:
### count lost packets and calculate PRR (APPROach 1)
hb_pkts = []
data_pkts_rb = []
board_index = [primary_board_ind, redundant_board_ind]
pkts_lost_pb = 0
pkts_lost_rb = 0
pb_reset_count = 0
rb_reset_count = 0
for i in board_index:
    each_board = data[i]
    if i == primary_board_ind:
        previous_id = 0
        for d in each_board:
            print(d[1])
            if d[1] - previous_id > 1:
                print(d[1], previous_id)
                pkts_lost_pb += (d[1] - previous_id - 1)
            if d[1] - previous_id < 0:
                pb_reset_count += 1
            previous_id = d[1]
    if i == redundant_board_ind:
        previous_id = 0
        data_pkts_rb = []
        for d in each_board:
            #print(d[0][0])
            if d[5][0] == -1:
                hb_pkts += [d]
            else:
                print(d[1])
                if d[1] - previous_id > 1:
                    if d[1] > 20 and d[1] < 47:
                        print(d[1], previous_id)
                        pkts_lost_rb += (d[1] - previous_id - 1)
                if d[1] - previous_id < 0:
                    pb_reset_count += 1
                data_pkts_rb += [d]
                previous_id = d[1]

### PRR with Redundancy (PB+RB)
total_pkts = 47 + 22             ### last packet of redundant board and 2nd session of PB
total_pkts_lost = pkts_lost_pb+pkts_lost_rb
prr_sn = ((total_pkts-total_pkts_lost)/total_pkts) * 100
print('PRR with Redundancy:', prr_sn)

### PRR without Redundancy (PB)
prr_pb = ((20+22-pkts_lost_pb)/total_pkts) * 100
print('PRR without Redundancy:', prr_pb)
                    

0
1
2
3
4
5
6
8
8 6
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
6
6 4
8
8 6
10
10 8
11
12
14
14 12
16
16 14
17
18
19
20
21
22
1
2
4
6
7
8
9
10
12
13
14
16
17
18
19
20
23
23 20
25
25 23
27
27 25
28
29
31
31 29
32
33
34
35
36
37
38
39
40
42
42 40
43
44
46
46 44
47
5
9
PRR with Redundancy: 81.15942028985508
PRR without Redundancy: 52.17391304347826


In [37]:
### PRR with Redundancy (PB+RB)
packet_num = []
num_pkts_sent = 0 
num_pkts_rx = 0
for d in data_pkts:
    if d[1] not in packet_num:
        packet_num += [d[1]]
num_pkts_sent = max(packet_num)
num_pkts_rx = len(packet_num)
prr_sn = (num_pkts_rx/num_pkts_sent) * 100
print('PRR of the sensor node:', prr_sn)

### PRR without Redundancy (PR)


PRR of the sensor node: 91.48936170212765


#### Efficiency of RB 


In [30]:
data[1]

[(94420780,
  0,
  31,
  31,
  -104,
  (585.0999755859375,
   0.0,
   23.304964065551758,
   1009.3259887695312,
   22.600000381469727,
   39.400001525878906,
   22.80000114440918,
   46.20000076293945,
   22.80000114440918,
   47.0,
   22.899999618530273,
   46.5,
   -157.0,
   0,
   0,
   0,
   94420780)),
 (94420780,
  1,
  61,
  61,
  -103,
  (601.4099731445312,
   0.0,
   22.413785934448242,
   1009.2979736328125,
   22.600000381469727,
   39.20000076293945,
   22.899999618530273,
   46.20000076293945,
   22.899999618530273,
   46.70000076293945,
   22.899999618530273,
   46.400001525878906,
   -44.0,
   0,
   0,
   1,
   94420780)),
 (94420780,
  2,
  91,
  91,
  -94,
  (595.6400146484375,
   0.0,
   22.071962356567383,
   1009.27734375,
   22.700000762939453,
   39.0,
   22.899999618530273,
   45.900001525878906,
   22.899999618530273,
   46.400001525878906,
   22.899999618530273,
   46.10000228881836,
   -44.0,
   0,
   0,
   2,
   94420780)),
 (94420780,
  3,
  121,
  121,
  -

In [31]:
data[2]

[(2750291925,
  0,
  62,
  62,
  -98,
  (-1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -26.0,
   0,
   0,
   0,
   2750291925)),
 (2750291925,
  0,
  122,
  122,
  -96,
  (-1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -26.0,
   0,
   0,
   0,
   2750291925)),
 (2750291925,
  0,
  299,
  299,
  -94,
  (-1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -29.0,
   0,
   0,
   0,
   2750291925)),
 (2750291925,
  0,
  359,
  359,
  -94,
  (-1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -27.0,
   0,
   0,
   0,
   2750291925)),
 (2750291925,
  0,
  419,
  419,
  -102,
  (-1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -1.0,
   -27.0,
   0,
   0,
   0,
   2750291925)),
 (2750291925,
  0,
  479,
  479,
  -

In [78]:
### efficiency of RB
# how many faulty pkts detected?    how many lost packets detected

hb_pkts = []
data_pkts = []

for d in data[redundant_board_ind]:
    #print(d[0][0])
    if d[5][0] == -1:
        hb_pkts += [d]
    else:
        data_pkts += [d]
        
faulty_detected_pkts = []
lost_detected_pkts = []
        
for d in data_pkts:
    if d[1] < 20 or d[1] > 47:   ### 20: last pkt by PB before failure, 47: last pkt by RB bfore resetting pkt count
        lost_detected_pkts += [d]
    else:
        faulty_detected_pkts += [d]

detection_rate = len(faulty_detected_pkts)/(47-20)*100
print('Detection rate for hard failure:', detection_rate)

Detection rate for hard failure: 77.77777777777779
